# Qaintellect training example 2: multiple qubits and measurement operators

In this example, we will demonstrate the optimisation of a quantum circuit with multiple qubits and measurement operators.

In [1]:
using Qaintessent
using Qaintellect
using LinearAlgebra
using Flux
using IterTools: ncycle

We create a parametrized quantum circuit with $N$ qubits. We then optimize these parameters using the Flux framework.

In [2]:
# construct parametrized circuit

N = 4

ry = RyGate(√2)
n = randn(FloatQ, 3)
n /= norm(n)
rg = RotationGate(0.2π, n)
@show(rg)

cgc = [
    circuit_gate(3, HadamardGate()),
    circuit_gate(2, RzGate(1.5π), (1, 4)), # controlled gate
    circuit_gate(2, 3, SwapGate()),
    circuit_gate(3, PhaseShiftGate(0.3)),
    circuit_gate(3, rg),
    circuit_gate(1, ry),
]
meas = [MeasurementOperator(Matrix{Float64}(I, 2^N, 2^N), Tuple(1:N)), MeasurementOperator(Hermitian(randn(ComplexF64, 2^N, 2^N)), Tuple(1:N))]

c = Circuit{N}(cgc, meas)

rg = RotationGate(Float32[-0.53703034, 0.05597703, -0.3213241])



    4 ————————•—————————————————————
              |                     
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |     |               
    2 ———————[Rz]———x———————————————
              |                     
    1 ————————•————————————————[Ry]—


In [3]:
# set up model
model(ψ) = dot([0.3, -1.2], c(ψ))

# create loss function
loss(x, y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

paras

Params([Float32[4.712389], Float32[0.3], Float32[-0.53703034, 0.05597703, -0.3213241], sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], ComplexF32[1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im, 1.0f0 + 0.0f0im], 16, 16), sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  16, 16, 16, 16, 16, 16, 16, 16, 16, 16], ComplexF32[-1.4366738f0 + 0.0f0im, 0.03560376f0 - 0.6054848f0im, -0.527578f0 - 0.27173388f0im, -1.3991385f0 - 1.1131895f0im, -1.1278617f0 - 0.37390763f0im, 0.75828916f0 - 0.37697834f0im, -0.17399871f0 + 0.68030006f0im, -0.9440648f0 - 0.098141216f0im, 0.24595958f0 + 0.6661913f0im, -0.4319537f0 - 0.27240527f0im  …  1.8054765f0 + 0.6945263f0im, 

In [4]:
# input quantum state
ψ = randn(ComplexQ, 2^N)
ψ /= norm(ψ)

# desired output
e = 0.65

# set up data for training
data = ncycle([(ψ, e)], 64)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

println("Initial model evaluation: $(model(ψ)), target: $e")

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: $(model(ψ)), target: $e")

Initial model evaluation: -0.5137484192848205, target: 0.65
loss(ψ, e) = 4.427609752931719
loss(ψ, e) = 1.1252841773625583e-5
loss(ψ, e) = 1.1828014123388259e-11
loss(ψ, e) = 2.1850610211407502e-12
Final model evaluation: 0.6499985218048095, target: 0.65


In [5]:
# check: Ry gate has not changed (frozen parameter)
@show(ry)

# on the other hand, parameters of general rotation gate are now different
@show(rg);

ry = RyGate(Float32[1.4142135])
rg = RotationGate(Float32[2.4036388, -1.0326958, -0.18446656])
